# Chapter 16. Coroutines

If Python books are any guide, [coroutines are] the most poorly documented, obscure, and apparently uesless feature of Python.

-- David Beazley (Python author)

A line such as `yiled item` produces a value that is received by the caller of `next(...)`, and it also gives way, suspending the execution of the generator so that the calleer may proceed until it's ready to consume another value by invoking `next()` again. The caller pull values from the generator.

A coroutinue is syntactically like a generator: just a function with the `yield` keyword in its body. However in routine, `yield` usually appears on the righ side of an expression (e.g., `datum = yield`), and it may or may not produce a value - if there is no expression after yield keyword, the generator yields None. The coroutine may receive data from the caller, which uses `.send(datum)` instead of `next(...)` to feed the coroutine. Usually, the caller pushes values into the coroutine.

Regardless of the flow of data, `yield` is a control flow device that can be used to implement cooperative multitasking: each coroutine yields control to a central schedular so that other coroutines can be activated.

When you start thinking of yield primarily in terms of control flow, you have the midset to understand coroutines.

Following the evolution of coroutines in Python helps us understand their features in stages of increasing functionality and complexity.

After a brief overview of how generators were enable to act as a coroutine, we jump to the core of chapter:

* The behavior and states of a generator operating as a coroutine
* Priming a coroutine automatically with a decorator
* **How the caller can control a coroutine through the `.close()` and `.throw(...)` methods of the generator object**
* **How coroutine can return values upon termination**
* **Usage and semantics of the new `yield from` syntax**
* A use case: coroutine for managing concurrent activities in a simulation

## How Coroutines Evolved from Generators

**The infrastructure for coroutines appeared in [PEP342 - Coroutine via Enhenced Generators](https://www.python.org/dev/peps/pep-0342/) implemented in Python 2.5: since then, the yield keyword can be used in an expression, and the `.send(value)` method was added to the generator API. Using `.send(...)`, the caller of the generator can post data that then becomes the value of the yield expression inside the generator function. This allows a generator to be used as a coroutine: a procedure that collaborates with the caller, yielding and receiving values from the caller.**

In addition to `.send(...)` PEP 342 also added `.throw(...)` and `.close()` methods that respectively allow the caller to throw an exception to be handled inside the generator, and to terminate it.

The latest evolutionary step for coroutines came with [PEP 380 - Syntax for Delegating to a Subgenerator](https://www.python.org/dev/peps/pep-0380/). PEP380 made two syntax changes to generator functions, to make them more useful as coroutines:

* A generator can now return a value; providing value to the return statement inside a generator raised a SyntaxError
* The `yield from` syntax enables complex generators to be refactored into smaller, nested generators while avoiding a lot of boilerplate code previously required for a generator to delegate to subgenerators.

## Basic Behavior of a Generator Used as a Coroutine

In [13]:
def simple_coroutine():
    print('-> coroutine started')
    # yield is used in a expresstion; when the coroutine is designed
    # just to receive data from the client it yield None -- this is
    # implicit because there is no expression to the right of the
    # yield keyword.
    x = yield
    print('-> coroutine received: ', x)

In [14]:
my_cor = simple_coroutine()
my_cor

<generator object simple_coroutine at 0x10ad75228>

In [15]:
next(my_cor)

-> coroutine started


In [16]:
my_cor.send(42)
# control flow off the end of the coroutine body, which prompts the
# generator machinery to raise StopIteration, as usual.

-> coroutine received:  42


StopIteration: 

A coroutine can be in one of the four stats. You can determine the current state using the `inspect.getgeneratorstate()`function, which returns one of these strings:

* GEN_CREATED, Waiting to start execution
* GEN_RUNNING, Currently being executed by the interpreter
* GEN_SUSPENDED, Currently suspended at a yield expression
* GEN_CLOSED, Execution has completed

In [19]:
import inspect

In [21]:
inspect.getgeneratorstate(my_cor)

'GEN_CLOSED'

In [22]:
def simple_coroutine(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b=', b)
    c = yield b
    print('-> Received: c=', c)

In [23]:
my_coro2 = simple_coroutine(14)

In [24]:
inspect.getgeneratorstate(my_coro2)

'GEN_CREATED'

In [25]:
next(my_coro2)

-> Started: a = 14


14

In [26]:
inspect.getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [27]:
my_coro2.send(28)

-> Received: b= 28


28

In [29]:
my_coro2.send(99)

-> Received: c= 99


StopIteration: 

In [30]:
inspect.getgeneratorstate(my_coro2)

'GEN_CLOSED'

## Example: Coroutine to Compute a Running Average

In [31]:
def averager():
    total = 0
    count = 0
    average = 0

    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [33]:
coro_avg = averager()
next(coro_avg)

0

In [34]:
coro_avg.send(10)

10.0

In [35]:
coro_avg.send(30)

20.0

In [36]:
coro_avg.send(5)

15.0

## Decorator for Couroutine Priming

You can't do much with a coroutine without priming it: we must always remember to call `next(my_coro)` before `my_coro.send()`. To make corotine usage more convenient, a priming decorator is sometimes used.

In [37]:
import functools

In [42]:
def coroutine(func):
    @functools.wraps(func)
    def primer(*args, **kwargs):
         # call the decorated function to get a generator object
        gen = func(*args, **kwargs)
        next(gen) # prime the generator
        return gen
    return primer

In [43]:
@coroutine
def averager():
    average = 0
    total = 0
    count = 0
    while True:
        term = yield average 
        total += term
        count += 1
        average = total / count

In [44]:
avg = averager()

In [45]:
avg.send(10)

10.0

In [46]:
avg.send(20)

15.0

## Coroutine Termination and Exception Handling

An unhandled exception within a coroutine propagates to the caller of the next or send that triggered it.

In [52]:
avg = averager()

In [53]:
avg.send(10)
avg.send(20)

15.0

In [54]:
# Because the exception was not  handled in the coroutine, it 
# terminated. Any attempt to reactive it will raise StopIteration.
avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

You can use send with some sentinel value that tells the coroutine to exit. Another sentinel value I've seen used is `StopIteration` the calss itself, not an instance of it. In other words, using it like `my_coro.send(StopIteration)`

* generator.throw(exc_type[, exc_val[, traceback]])
    * Causes the yield exception is handled by the generator was paused to raise the exception given. If the exception is handled by the generator, flow advances to the next yield, and the value yilded becomes the value of the generator.throw call. If the exception is not handled by the generator, it propagates to the context of the caller.
* generator.close()
    * Causes the yield expression where the generator was paused to raise a GeneratorExit exception. No error is reported to the caller if the generator does not handle that exception. When receiving a GeneratorExit, the generator must not yield a value, otherwise a RunTimeError is raised. If any other exception is raised by the generator, it propagates to the caller.

Let's see how close and throw control a coroutine.

In [62]:
class DemoException(Exception): pass

In [70]:
def demo_exc_handling():
    print('-> coroutine started')

    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine recevied: {!r}'.format(x))
            
    raise RuntimeError('This line should never run')

Activating and closing demo_exc_handling without an exception.

In [71]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [72]:
exc_coro.send(10)

-> coroutine recevied: 10


In [73]:
exc_coro.close()

In [74]:
inspect.getgeneratorstate(exc_coro)

'GEN_CLOSED'

Throwing DemoException into demo_exc_handling does not break it.

In [75]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [76]:
exc_coro.send(10)

-> coroutine recevied: 10


In [77]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [78]:
inspect.getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

Coroutine terminates if it can't handle an exception thrown into it.

In [79]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [80]:
exc_coro.send(10)

-> coroutine recevied: 10


In [81]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [82]:
inspect.getgeneratorstate(exc_coro)

'GEN_CLOSED'

## Returning a Value from a Coroutine

In [83]:
import collections

In [84]:
Result = collections.namedtuple('Result', 'count average')

In [85]:
def averager():
    total = 0
    count = 0
    average = 0
    
    while True:
        term = yield average
        if term is None:
            break
            
        total += term
        count += 1
        average = total / count
    
    return Result(count, average)

In [95]:
avg = averager()
next(avg)

0

In [96]:
avg.send(10)
avg.send(20)
avg.send(30)

20.0

In [97]:
inspect.getgeneratorstate(avg)

'GEN_SUSPENDED'

In [98]:
avg.send(None)

StopIteration: Result(count=3, average=20.0)

Sending None terminates the loop, causeing the coroutine to end by returning the result. As usually, the generator object raises StopIteration. The value attribute of the exception carries the value returned.

Catching StopIteration lets us get the value returned by averager.

In [100]:
coro_avg = averager()
next(coro_avg)

0

In [101]:
coro_avg.send(100)
coro_avg.send(20)
coro_avg.send(10)

43.333333333333336

In [102]:
try:
    coro_avg.send(None)
except StopIteration as e:
    result = e.value

In [103]:
result

Result(count=3, average=43.333333333333336)

In [104]:
inspect.getgeneratorstate(coro_avg)

'GEN_CLOSED'

This roundabout way of getting the return value from a coroutine makes more sense when we realize it was defined as part of PEP380, and the `yield from` construct handles it automatically by catching StopIteration internally.

In [105]:
help(StopIteration)

Help on class StopIteration in module builtins:

class StopIteration(Exception)
 |  Signal the end from iterator.__next__().
 |  
 |  Method resolution order:
 |      StopIteration
 |      Exception
 |      BaseException
 |      object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  value
 |      generator return value
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from Exception:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from BaseException:
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |

## Using yield from

The first thing to known about `yield from` is that it is a completely new language construct. When a generator gen calls `yield from subgen()`, the `subgen` take over and will yield values to the caller of gen; the caller will in effect drive subgen directly. Meanwhile gen will blocked, waiting until subgen terminates.

In [106]:
def gen():
    for char in 'AB':
        yield char
        
    for num in '12':
        yield num

In [107]:
list(gen())

['A', 'B', '1', '2']

Can be written as:

In [108]:
def gen():
    yield from 'AB'
    yield from '12'

In [110]:
list(gen())

['A', 'B', '1', '2']

Chaining iterables with yield from

In [111]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [112]:
s = 'ABC'
t = range(3)
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

**The first thing the `yield from x`  expression does with the x object is to call `iter(x)` to obtain an iterator from it. This mean that x can be any iterable.**

The real nature of `yield from` cannot be demonstrated with simple iterables; it requires the mind-expanding use of nested generators. That's why PEP380, which introduced `yield from`, is titled "Syntax for Delegating to a Subgenerator".

**The main feature of `yield from` is to open a bidirectional channel from the outermost caller to the innermost subgenerator, so that values can be sent and yield back and forth directly from them, and exceptions can be thrown all the way in without adding a lot of exception handling boilerplate code in the intermedia coroutines.** This is what enables coroutine delegation in a way that was not possible before.

* delegating generator - The generator function that contains `yield from <iterable>` expression.
* subgenerator - The generator obtains from the `<iterable>` part of the `yield from` expression.
* caller - PEP380 uses the term "caller" to refer to the client code that calls the delegating generator.

> PEP380 often uses the word "iterator" to refer to the subgenerator. That's confusing because the delegating generator is also an iterator. So I prefer to use the term subgenerator.

![](https://ss0.bdstatic.com/70cFvHSh_Q1YnxGkpoWK1HF6hhy/it/u=1042154715,4097037307&fm=15&gp=0.jpg)

While the deletating generator is suspended at `yield from`, the caller sends data directly to the subgenerator, which yields data backs to the caller. The delegating generator resumes when the subsgenerator returns and the interpreter raise StopIteration with the returned value attached.

In [114]:
import inspect
import collections

In [115]:
Result = collections.namedtuple('Result', 'count average')

In [132]:
# the subgenerator
def averager():
    total = 0.0
    count = 0
    average = None
    
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
        
    # The returned Result will be the value of the yield from
    # expression in grouper
    return Result(count, average)

In [156]:
# the delegating generator

def grouper(results, key):
    # Each iteration in this loop creats a new instance of 
    # averager; each is a generator object operating as a 
    # coroutine
    while True:
        # Whenever grouper is sent a value, it's piped into the
        # averager instance by the yield from.grouper will be
        # suspended here as long as the averager instance is consuming
        # values sent by the client. When an averager instance runs
        # to the end, the value it returns is bound to `reuslts[key]`.
        # The while loop then proceeds to create another averager
        # instance to consume more values.
        results[key] = yield from averager()

In [151]:
# the client code, a.k.a. the caller
def main(data):
    resutls = {}
    
    for key, values in data.items():

        group = grouper(resutls, key)
        # prime the coroutine.
        next(group)
        for value in values:
            # Sending each value into the grouper. That value ends up
            # in the term= yield line of averager;
            # grouper never has a chance to see it.
            group.send(value)

        # Sending None into grouper cause the current averager
        # instance to terminate, and allows grouper to run again,
        # which creates another averager for the next group of values.
        group.send(None) # important!
        
    print(resutls)

In [153]:
data= {
    'girls;kg': [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m': [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

In [157]:
main(data)

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}


> If a subgenerator never terminates, the delegating generator will be suspended forever at the yield from. This will be not prevent program from making progress because the yield from transfer control to the cliet code.

## The Meaning of yield from

The approved version of PEP380 explains the behavior of yield from in six points in the [Proposal section](https://www.python.org/dev/peps/pep-0380/%23proposal)

* Any values that subgenerator yields are passed directly to the caller of the delegating generator(i.e., the client code)
* Any values sent to the delegating generator using `send()` are passed directly to the subgenerator. If the send value is None, the subgenerator's `__next__()`  method is called. If the send value is not None, the subgenerator's `send()` method is called. If the call raises StopIteration, the delegating generator is resumed. Any other exception is propagated to the delegating generator.
* `return expr` in a generator (or subgenerator) causes StopIteration to be raised upon exit from the generator.
* The value of the yield from expression is the first argument to the StopIteration exception raised by the subgenerator when it terminates.

The other two features of yield from have to do with exceptions and termination:

* Exceptions other than `GeneratorExit` thrown into delegating generator are passed to the `throw()` method of the subgenerator. If the call raises StopIteration, the delegating generator is resumed. Any other exception is propagated to the delegating generator.
* If a GeneratorExit exception is thrown into the delegating generator, or the `close()` method of the delegating generator is called, the `close()` method of the subgenerator is called if it has one. If this call results in an exception, it is propagated to the delegating generator. Otherwise, GeneratorExit is raised in the delegating generator.

Simplified pseudocode equivalent to the statement `RESULT = yield from EXPR` in the delegating generator (this covers the simplet case: .throw(...), and .close() are not supported; the only exception handled is StopIteration).

```python

_i = iter(EXPR)

try:
    _y = next(_i)
except StopIteration as _e:
    _r = _e.value
else:
    while 1:
        _s = yield _y

        try:
            _i.send(_s)
        except StopIteration as _e:
            _r = _e.value
            break

RESULT = _r

```

```python

# The EXPR can be any iterable, because iter() is applied to get an iterator _i(this is th e subgenerator)
_i = iter(EXPR)

try:
    # The subgenerator is primed; the result is stored to the first yield value _y
    _y = next(_i)
except StopIteration as _e:
    # If StopIteration was raised, extract the value attribute from the exception and assign it to _r: this is the RESULT in the simplest case.
    _r = _e.value
else:
    # Whiel the loop is running, the delegating generator is blocked, operating just as a channel between the caller and the subgenerator.
    while 1:
        
        # Wait for a value _s sent by the caller.
        _s = yield _y
        try:
            # Try to advance the subgenerator, forwarding the _s sent by the caller.
            _y = _i.send(_s)
        
            # If the subgenerator raised StopIteration, get the value, assign to _r, and exit the loop, resuming the delegating generator.
        except StopIteration as _e:
            _r = _e.value
            break

# _r is the RESULT: the value of the whole yield from expression.
RESULT = _r

```

* _i(iterator) - The subgenerator
* _y(yielded) - A value yielded from the subgenerator
* _r(reuslt) - The eventual result(i.e., the value of the yield from expression when the subgenerator ends)
* _s(sent) - A value sent by the caller to the delegating generator, which is forwarded to the subgenerator
* _e(exception) - An exception(always an instance of StopIteration in this simplified pseudocode)

The reality is more compliated, because of the need to handle `.throw(...)` and `.close()` calls from the client, which must be passed into the subgenerator. Also the  subgenerator may be a plain iterator that does not support `.throw(...)` or `.close()`, so this must be handled by the `yield from` logic. If the subgenerator does implement those methods, inside the subgenerator both methods cause exceptions to be raised, which must be handled by the yield from machinary as well. The subgenerator may also throw exceptions of its own, and this must be also be dealt with in the `yield from` implemenation. Finally, as an optimization, if the caller calls next(...) or .send(None), both are forwarded as a next(...) call on the subgenerator.

Psuedocode equivalent to the statement `RESULT = yield from EXPR` in the delegating generator

In [1]:
import sys

```python
_i = iter(EXPR)

try:
    _y = next(_i)
except StopIteration as _e:
    _r = _e.value
else:
    while 1:
        try:
            _s = yield _y
        except GeneratorExit as _e:
            try:
                _m = _i.close
            except AttributeError:
                pass
            else:
                _m()
            raise _e
        except BaseException as _e:
            _x = sys.exc_info()
            try:
                _m = _i.throw
            except AttributeError:
                raise _e
            else:
                try:
                    _y = _m(*_x)
                except StopIteration as _e:
                    _r = _e.value
                    break
        else:
            try:
                if _s is None:
                    _y = next(_i)
                else:
                    _y = _i.send(_s)
            except StopIteration as _e:
                _r = _e.value
                break

RESULT = _r
```

## Chapter Summary

Guido van Rossum wrote there are three different styles of code you can write using generators:

    There's the traditional "pull" style (iterators), "push" style (like the averging exampole), and then there are "tasks".

The running average example demonstrated a common use for a coroutine: as accumulator processing items sent to it. We saw a decorator can be applied to prime a coroutine, making it more convenient to use in some cases.

In PEP380, we saw how the statement `return the_result` in a generator raises `StopIteration(the_result)`, allowing the caller to retrieve the_result from the value attribute of the exception. This is a rather cumbersome way to retrieve coroutine result, but it's handled automatically by the `yield from` syntax introduced in PEP 380.

The coverage of `yield from` started with trival example using simple iterables, then moved to an example the three main components of any significant use of `yield from`: the delegating generator (defined by the use of yield from in its body), the subgenerator activated by `yield from`, and the client code that actually drives the whole setup by `yield from` in the delegating generator.